In [26]:
import os
import sys
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from tqdm.notebook import tqdm
from collections import namedtuple
from collections import defaultdict
from pprint import pprint
import pickle

# Configure display options
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)
%config InlineBackend.figure_format = 'retina'

# Add project directory to sys.path
# pdir = os.path.abspath(os.path.join(os.getcwd(), '.'))
# sys.path.insert(1, pdir)
# from myutils import *

# Other module imports
import ast, math, swifter, csv, json, itertools as it, portion as P

# Set plot style
# plt.style.use('ggplot')

In [30]:
filenames = []

root_dir = "/Users/jackbedford/Desktop/MOXA/Code/data"
date_dirs = [s for s in os.listdir(root_dir) if s != '.DS_Store' and not s.endswith('.zip')]
date_dirs = [os.path.join(root_dir, s) for s in sorted(date_dirs)]
date_dirs = date_dirs[:-7]
date_dirs

for date_dir in date_dirs:
    # print(date_dir)
    exp_dirs = [s for s in os.listdir(date_dir) if os.path.isdir(os.path.join(date_dir, s)) and s != '.DS_Store']
    exp_dirs = [os.path.join(date_dir, s) for s in sorted(exp_dirs)]
    for exp_dir in exp_dirs:
        # print(exp_dir)
        dev_dirs = [s for s in os.listdir(exp_dir) if os.path.isdir(os.path.join(exp_dir, s)) and s != '.DS_Store']
        dev_dirs = [os.path.join(exp_dir, s) for s in sorted(dev_dirs)]
        for dev_dir in dev_dirs:
            # print(dev_dir)
            trip_dirs = [s for s in os.listdir(dev_dir) if os.path.isdir(os.path.join(dev_dir, s)) and s != '.DS_Store']
            trip_dirs = [os.path.join(dev_dir, s, 'data') for s in sorted(trip_dirs)]
            for trip_dir in trip_dirs:
                # print(trip_dir)
                rrc_file = [os.path.join(trip_dir, s) for s in os.listdir(trip_dir) if s.endswith('rrc.csv')][0]
                if os.path.isfile(rrc_file):
                    filenames.append(rrc_file)
                
pprint(filenames, sort_dicts=False)

['/Users/jackbedford/Desktop/MOXA/Code/data/2023-03-16/Bandlock_Udp_B1_B3_B7_B8_RM500Q/qc00/#01/data/diag_log_qc00_2023-03-16_19-23-29_rrc.csv',
 '/Users/jackbedford/Desktop/MOXA/Code/data/2023-03-16/Bandlock_Udp_B1_B3_B7_B8_RM500Q/qc00/#02/data/diag_log_qc00_2023-03-16_19-32-43_rrc.csv',
 '/Users/jackbedford/Desktop/MOXA/Code/data/2023-03-16/Bandlock_Udp_B1_B3_B7_B8_RM500Q/qc00/#03/data/diag_log_qc00_2023-03-16_19-38-36_rrc.csv',
 '/Users/jackbedford/Desktop/MOXA/Code/data/2023-03-16/Bandlock_Udp_B1_B3_B7_B8_RM500Q/qc00/#04/data/diag_log_qc00_2023-03-16_19-47-07_rrc.csv',
 '/Users/jackbedford/Desktop/MOXA/Code/data/2023-03-16/Bandlock_Udp_B1_B3_B7_B8_RM500Q/qc00/#05/data/diag_log_qc00_2023-03-16_19-55-14_rrc.csv',
 '/Users/jackbedford/Desktop/MOXA/Code/data/2023-03-16/Bandlock_Udp_B1_B3_B7_B8_RM500Q/qc00/#06/data/diag_log_qc00_2023-03-16_20-03-44_rrc.csv',
 '/Users/jackbedford/Desktop/MOXA/Code/data/2023-03-16/Bandlock_Udp_B1_B3_B7_B8_RM500Q/qc00/#07/data/diag_log_qc00_2023-03-16_20-0

In [33]:
for filename in filenames:
    df = pd.read_csv(filename)
    if sum((df['scgFailureInformationNR-r15'] == '1') | (df['scgFailureInformationNR-r15'] == 1)) > 0 and \
        sum((df['rrcConnectionReestablishmentComplete'] == '1') | (df['rrcConnectionReestablishmentComplete'] == 1)) > 0 and \
        sum((df['rrcConnectionReestablishmentReject'] == '1') | (df['rrcConnectionReestablishmentReject'] == 1)):
            print(filename)
            display(df[(df['scgFailureInformationNR-r15'] == '1') | (df['scgFailureInformationNR-r15'] == 1) | 
                       (df['rrcConnectionReestablishmentComplete'] == '1') | (df['rrcConnectionReestablishmentComplete'] == 1) |
                       (df['rrcConnectionReestablishmentReject'] == '1') | (df['rrcConnectionReestablishmentReject'] == 1)]) 

/Users/jackbedford/Desktop/MOXA/Code/data/2023-05-15/Bandlock_UDP_8_Phone/sm02/#04/data/diag_log_sm02_2023-05-15_15-54-14_rrc.csv


,Timestamp,type_id,PCI,UL_DL,Freq,DL frequency,UL frequency,DL bandwidth,UL bandwidth,Cell Identity,TAC,Band ID,MCC,MNC,lte-measurementReport,nr-measurementReport,measId,MeasResultEUTRA,physCellId,MeasResultServFreqNR-r15,pci-r15,MeasResultNR,physCellId.1,measResultServingCell,physCellId.2,MeasResultCellNR-r15,pci-r15.1,lte-MeasObjectToAddMod,nr-MeasObjectToAddMod,measObjectId,measObject,carrierFreq,carrierFreq-r15,ssbFrequency,lte-ReportConfigToAddMod,lte-reportConfigId,triggerType,lte-eventId,lte-parameter,nr-ReportConfigToAddMod,nr-reportConfigId,reportType,nr-eventId,nr-parameter,lte-measIdToRemoveList,lte-MeasIdToAddMod,nr-MeasIdToAddMod,rrcConnectionReconfiguration,rrcConnectionReconfigurationComplete,RRCReconfiguration,RRCReconfigurationComplete,rrcConnectionReestablishmentRequest,physCellId.3,reestablishmentCause,rrcConnectionReestablishment,rrcConnectionReestablishmentComplete,rrcConnectionReestablishmentReject,rrcConnectionRequest,rrcConnectionSetup,rrcConnectionSetupComplete,securityModeCommand,securityModeComplete,rrcConnectionRelease,systemInformationBlockType1,nr-Config-r15: release (0),nr-Config-r15: setup (1),dualConnectivityPHR: release (0),dualConnectivityPHR: setup (1),scgFailureInformationNR-r15,failureType-r15,lte_targetPhysCellId,dl-CarrierFreq,lte-rrc.t304,nr_physCellId,nr-rrc.t304,sCellToReleaseList-r10,SCellIndex-r10,SCellToAddMod-r10,SCellIndex-r10.1,physCellId-r10,dl-CarrierFreq-r10
136,2023-05-15 07:55:17.605807,LTE_RRC_OTA_Packet,73,UL,3400.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0,0,0,0.0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
335,2023-05-15 07:57:31.374245,LTE_RRC_OTA_Packet,197,UL,3050.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0,0,0,0.0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,synchReconfigFailureSCG (3),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
512,2023-05-15 07:58:27.832732,LTE_RRC_OTA_Packet,178,-,3050.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0,0,0,0.0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
551,2023-05-15 07:58:32.406400,LTE_RRC_OTA_Packet,130,UL,3400.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0,0,0,0.0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0


/Users/jackbedford/Desktop/MOXA/Code/data/2023-05-24/Bandlock_UDP_8_Phone/sm05/#03/data/diag_log_sm05_2023-05-24_15-10-53_rrc.csv


,Timestamp,type_id,PCI,UL_DL,Freq,DL frequency,UL frequency,DL bandwidth,UL bandwidth,Cell Identity,TAC,Band ID,MCC,MNC,lte-measurementReport,nr-measurementReport,measId,MeasResultEUTRA,physCellId,MeasResultServFreqNR-r15,pci-r15,MeasResultNR,physCellId.1,measResultServingCell,physCellId.2,MeasResultCellNR-r15,pci-r15.1,lte-MeasObjectToAddMod,nr-MeasObjectToAddMod,measObjectId,measObject,carrierFreq,carrierFreq-r15,ssbFrequency,lte-ReportConfigToAddMod,lte-reportConfigId,triggerType,lte-eventId,lte-parameter,nr-ReportConfigToAddMod,nr-reportConfigId,reportType,nr-eventId,nr-parameter,lte-measIdToRemoveList,lte-MeasIdToAddMod,nr-MeasIdToAddMod,rrcConnectionReconfiguration,rrcConnectionReconfigurationComplete,RRCReconfiguration,RRCReconfigurationComplete,rrcConnectionReestablishmentRequest,physCellId.3,reestablishmentCause,rrcConnectionReestablishment,rrcConnectionReestablishmentComplete,rrcConnectionReestablishmentReject,rrcConnectionRequest,rrcConnectionSetup,rrcConnectionSetupComplete,securityModeCommand,securityModeComplete,rrcConnectionRelease,systemInformationBlockType1,nr-Config-r15: release (0),nr-Config-r15: setup (1),dualConnectivityPHR: release (0),dualConnectivityPHR: setup (1),scgFailureInformationNR-r15,failureType-r15,lte_targetPhysCellId,dl-CarrierFreq,lte-rrc.t304,nr_physCellId,nr-rrc.t304,sCellToReleaseList-r10,SCellIndex-r10,SCellToAddMod-r10,SCellIndex-r10.1,physCellId-r10,dl-CarrierFreq-r10
48,2023-05-24 07:11:39.244432,LTE_RRC_OTA_Packet,178,-,1750.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0,0,0,0.0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
95,2023-05-24 07:11:45.600642,LTE_RRC_OTA_Packet,122,UL,1750.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0,0,0,0.0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
168,2023-05-24 07:12:05.759109,LTE_RRC_OTA_Packet,130,UL,1750.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,0.0,0,0.0,0.0,1.0,130@11@22,1.0,3.0,0.0,0,0.0,0.0,0,0,0,0.0,1.0,0.0,0,0,0,0,0.0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,synchReconfigFailureSCG (3),0.0,0.0,0.0,22.0,0.0,0.0,0.0,0.0,0,0,0
231,2023-05-24 07:12:11.807696,LTE_RRC_OTA_Packet,11,UL,1750.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0,0,0,0.0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0


/Users/jackbedford/Desktop/MOXA/Code/data/2023-06-15/Bandlock_All_B3_B7_B8_Dupl_Phone/sm02/#02/data/diag_log_sm02_2023-06-15_15-39-15_rrc.csv


,Timestamp,type_id,PCI,UL_DL,Freq,DL frequency,UL frequency,DL bandwidth,UL bandwidth,Cell Identity,TAC,Band ID,MCC,MNC,lte-measurementReport,nr-measurementReport,measId,MeasResultEUTRA,physCellId,MeasResultServFreqNR-r15,pci-r15,MeasResultNR,physCellId.1,measResultServingCell,physCellId.2,MeasResultCellNR-r15,pci-r15.1,lte-MeasObjectToAddMod,nr-MeasObjectToAddMod,measObjectId,measObject,carrierFreq,carrierFreq-r15,ssbFrequency,lte-ReportConfigToAddMod,lte-reportConfigId,triggerType,lte-eventId,lte-parameter,nr-ReportConfigToAddMod,nr-reportConfigId,reportType,nr-eventId,nr-parameter,lte-measIdToRemoveList,lte-MeasIdToAddMod,nr-MeasIdToAddMod,rrcConnectionReconfiguration,rrcConnectionReconfigurationComplete,RRCReconfiguration,RRCReconfigurationComplete,rrcConnectionReestablishmentRequest,physCellId.3,reestablishmentCause,rrcConnectionReestablishment,rrcConnectionReestablishmentComplete,rrcConnectionReestablishmentReject,rrcConnectionRequest,rrcConnectionSetup,rrcConnectionSetupComplete,securityModeCommand,securityModeComplete,rrcConnectionRelease,systemInformationBlockType1,nr-Config-r15: release (0),nr-Config-r15: setup (1),dualConnectivityPHR: release (0),dualConnectivityPHR: setup (1),scgFailureInformationNR-r15,failureType-r15,lte_targetPhysCellId,dl-CarrierFreq,lte-rrc.t304,nr_physCellId,nr-rrc.t304,sCellToReleaseList-r10,SCellIndex-r10,SCellToAddMod-r10,SCellIndex-r10.1,physCellId-r10,dl-CarrierFreq-r10
107,2023-06-15 07:40:20.926274,LTE_RRC_OTA_Packet,73,UL,1750.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
402,2023-06-15 07:43:11.196380,LTE_RRC_OTA_Packet,11,UL,1750.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,0.0,0,0.0,0.0,1.0,6@3,1.0,11.0,0.0,0,0.0,0.0,0,0,0,0,1.0,0.0,0,0,0,0,0.0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,synchReconfigFailureSCG (3),0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0,0.0
451,2023-06-15 07:43:23.935594,LTE_RRC_OTA_Packet,130,-,1400.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0.0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0


In [35]:
for filename in filenames:
    df = pd.read_csv(filename)
    if sum((df['scgFailureInformationNR-r15'] == '1') | (df['scgFailureInformationNR-r15'] == 1)) > 0 and \
        sum((df['rrcConnectionReestablishmentComplete'] == '1') | (df['rrcConnectionReestablishmentComplete'] == 1)) > 0 and \
        sum((df['rrcConnectionReestablishmentReject'] == '1') | (df['rrcConnectionReestablishmentReject'] == 1)):
            print(filename)
            display(df[(df['scgFailureInformationNR-r15'] == '1') | (df['scgFailureInformationNR-r15'] == 1)])
                    #    (df['rrcConnectionReestablishmentComplete'] == '1') | (df['rrcConnectionReestablishmentComplete'] == 1) |
                    #    (df['rrcConnectionReestablishmentReject'] == '1') | (df['rrcConnectionReestablishmentReject'] == 1)]) 

/Users/jackbedford/Desktop/MOXA/Code/data/2023-05-15/Bandlock_UDP_8_Phone/sm02/#04/data/diag_log_sm02_2023-05-15_15-54-14_rrc.csv


,Timestamp,type_id,PCI,UL_DL,Freq,DL frequency,UL frequency,DL bandwidth,UL bandwidth,Cell Identity,TAC,Band ID,MCC,MNC,lte-measurementReport,nr-measurementReport,measId,MeasResultEUTRA,physCellId,MeasResultServFreqNR-r15,pci-r15,MeasResultNR,physCellId.1,measResultServingCell,physCellId.2,MeasResultCellNR-r15,pci-r15.1,lte-MeasObjectToAddMod,nr-MeasObjectToAddMod,measObjectId,measObject,carrierFreq,carrierFreq-r15,ssbFrequency,lte-ReportConfigToAddMod,lte-reportConfigId,triggerType,lte-eventId,lte-parameter,nr-ReportConfigToAddMod,nr-reportConfigId,reportType,nr-eventId,nr-parameter,lte-measIdToRemoveList,lte-MeasIdToAddMod,nr-MeasIdToAddMod,rrcConnectionReconfiguration,rrcConnectionReconfigurationComplete,RRCReconfiguration,RRCReconfigurationComplete,rrcConnectionReestablishmentRequest,physCellId.3,reestablishmentCause,rrcConnectionReestablishment,rrcConnectionReestablishmentComplete,rrcConnectionReestablishmentReject,rrcConnectionRequest,rrcConnectionSetup,rrcConnectionSetupComplete,securityModeCommand,securityModeComplete,rrcConnectionRelease,systemInformationBlockType1,nr-Config-r15: release (0),nr-Config-r15: setup (1),dualConnectivityPHR: release (0),dualConnectivityPHR: setup (1),scgFailureInformationNR-r15,failureType-r15,lte_targetPhysCellId,dl-CarrierFreq,lte-rrc.t304,nr_physCellId,nr-rrc.t304,sCellToReleaseList-r10,SCellIndex-r10,SCellToAddMod-r10,SCellIndex-r10.1,physCellId-r10,dl-CarrierFreq-r10
335,2023-05-15 07:57:31.374245,LTE_RRC_OTA_Packet,197,UL,3050.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0,0,0,0.0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,synchReconfigFailureSCG (3),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0


/Users/jackbedford/Desktop/MOXA/Code/data/2023-05-24/Bandlock_UDP_8_Phone/sm05/#03/data/diag_log_sm05_2023-05-24_15-10-53_rrc.csv


,Timestamp,type_id,PCI,UL_DL,Freq,DL frequency,UL frequency,DL bandwidth,UL bandwidth,Cell Identity,TAC,Band ID,MCC,MNC,lte-measurementReport,nr-measurementReport,measId,MeasResultEUTRA,physCellId,MeasResultServFreqNR-r15,pci-r15,MeasResultNR,physCellId.1,measResultServingCell,physCellId.2,MeasResultCellNR-r15,pci-r15.1,lte-MeasObjectToAddMod,nr-MeasObjectToAddMod,measObjectId,measObject,carrierFreq,carrierFreq-r15,ssbFrequency,lte-ReportConfigToAddMod,lte-reportConfigId,triggerType,lte-eventId,lte-parameter,nr-ReportConfigToAddMod,nr-reportConfigId,reportType,nr-eventId,nr-parameter,lte-measIdToRemoveList,lte-MeasIdToAddMod,nr-MeasIdToAddMod,rrcConnectionReconfiguration,rrcConnectionReconfigurationComplete,RRCReconfiguration,RRCReconfigurationComplete,rrcConnectionReestablishmentRequest,physCellId.3,reestablishmentCause,rrcConnectionReestablishment,rrcConnectionReestablishmentComplete,rrcConnectionReestablishmentReject,rrcConnectionRequest,rrcConnectionSetup,rrcConnectionSetupComplete,securityModeCommand,securityModeComplete,rrcConnectionRelease,systemInformationBlockType1,nr-Config-r15: release (0),nr-Config-r15: setup (1),dualConnectivityPHR: release (0),dualConnectivityPHR: setup (1),scgFailureInformationNR-r15,failureType-r15,lte_targetPhysCellId,dl-CarrierFreq,lte-rrc.t304,nr_physCellId,nr-rrc.t304,sCellToReleaseList-r10,SCellIndex-r10,SCellToAddMod-r10,SCellIndex-r10.1,physCellId-r10,dl-CarrierFreq-r10
168,2023-05-24 07:12:05.759109,LTE_RRC_OTA_Packet,130,UL,1750.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,0.0,0,0.0,0.0,1.0,130@11@22,1.0,3.0,0.0,0,0.0,0.0,0,0,0,0.0,1.0,0.0,0,0,0,0,0.0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,synchReconfigFailureSCG (3),0.0,0.0,0.0,22.0,0.0,0.0,0.0,0.0,0,0,0


/Users/jackbedford/Desktop/MOXA/Code/data/2023-06-15/Bandlock_All_B3_B7_B8_Dupl_Phone/sm02/#02/data/diag_log_sm02_2023-06-15_15-39-15_rrc.csv


,Timestamp,type_id,PCI,UL_DL,Freq,DL frequency,UL frequency,DL bandwidth,UL bandwidth,Cell Identity,TAC,Band ID,MCC,MNC,lte-measurementReport,nr-measurementReport,measId,MeasResultEUTRA,physCellId,MeasResultServFreqNR-r15,pci-r15,MeasResultNR,physCellId.1,measResultServingCell,physCellId.2,MeasResultCellNR-r15,pci-r15.1,lte-MeasObjectToAddMod,nr-MeasObjectToAddMod,measObjectId,measObject,carrierFreq,carrierFreq-r15,ssbFrequency,lte-ReportConfigToAddMod,lte-reportConfigId,triggerType,lte-eventId,lte-parameter,nr-ReportConfigToAddMod,nr-reportConfigId,reportType,nr-eventId,nr-parameter,lte-measIdToRemoveList,lte-MeasIdToAddMod,nr-MeasIdToAddMod,rrcConnectionReconfiguration,rrcConnectionReconfigurationComplete,RRCReconfiguration,RRCReconfigurationComplete,rrcConnectionReestablishmentRequest,physCellId.3,reestablishmentCause,rrcConnectionReestablishment,rrcConnectionReestablishmentComplete,rrcConnectionReestablishmentReject,rrcConnectionRequest,rrcConnectionSetup,rrcConnectionSetupComplete,securityModeCommand,securityModeComplete,rrcConnectionRelease,systemInformationBlockType1,nr-Config-r15: release (0),nr-Config-r15: setup (1),dualConnectivityPHR: release (0),dualConnectivityPHR: setup (1),scgFailureInformationNR-r15,failureType-r15,lte_targetPhysCellId,dl-CarrierFreq,lte-rrc.t304,nr_physCellId,nr-rrc.t304,sCellToReleaseList-r10,SCellIndex-r10,SCellToAddMod-r10,SCellIndex-r10.1,physCellId-r10,dl-CarrierFreq-r10
402,2023-06-15 07:43:11.196380,LTE_RRC_OTA_Packet,11,UL,1750.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,0.0,0,0.0,0.0,1.0,6@3,1.0,11.0,0.0,0,0.0,0.0,0,0,0,0,1.0,0.0,0,0,0,0,0.0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,synchReconfigFailureSCG (3),0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0,0.0
